<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Sudoku
</p><br>
<strong>The ultimate spoiler for a time-wasting game ;)</strong>
<p>Possible improvements:</p>
- JS front-end
- randomly create a new game
- identify the level of difficulty of a game

<img src="https://automatetheboringstuff.com/images/000037.jpg">

### Import or create grid

In [26]:
import pandas as pd

In [27]:
#By default, the grid is converted to a table.
# dict_grid={
#     'A':[5,6,'',8,4,7,'','',''],
#     'B':[3,'',9,'','','',6,'',''],
#     'C':['','',8,'','','','','',''],
#     'D':['',1,'','',8,'','',4,''],
#     'E':[7,9,'',6,'',2,'',1,8],
#     'F':['',5,'','',3,'','',9,''],
#     'G':['','','','','','',2,'',''],
#     'H':['','',6,'','','',8,'',7],
#     'I':['','','',3,1,6,'',5,9]
    
# }
# grid=pd.DataFrame(data=dict_grid)
# grid

In [28]:
#Alternative: Open a template for user input. Save the file when done inputting a sudoku grid.
#This takes a while as it will open Excel
# import csv
# import subprocess

# with open('sudoku.csv', 'w',newline='') as csvfile:
#     filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#     filewriter.writerow(['',0,1,2,3,4,5,6,7,8])
#     for i in range(0,9):
#         filewriter.writerow([i])
# subprocess.call("explorer sudoku.csv")

In [29]:
grid=pd.read_csv('sudoku1.csv',index_col=0)
grid=grid.apply(pd.to_numeric).fillna('')
grid

,0,1,2,3,4,5,6,7,8
0,,,,2,,,,,4
1,,,,3,,,,,5
2,9,5,,,,,,2,
3,2,,5,1,,6,,7,9
4,,6,,,,,,3,
5,4,,7,9,,3,6,,2
6,,2,,,,,,4,3
7,6,,,,1,,,,
8,8,,,,9,,,,


### Define variables for the 3x3 blocks
<p>Insert into a function: to be recalculated every time a new value is entered in the grid</p>

In [30]:
#example:
upper_left=grid.iloc[:3,:3]
#the subgrids will be numbered from 0 to 8, from top-left to bottom-right. upper_left corresponds to sub_grids[0]
#remove the empty cells, we need blocks only to generate a set of digits.
def content_blocks(grid):
    blocks=[]
    i=1
    while i <10:
        col=0
        while col<9:
            if i in [1,2,3]:
                row=0
            elif i in [4,5,6]:
                row=3
            elif i in [7,8,9]:
                row=6
            sub=grid.iloc[row:row+3,col:col+3]
            sub = set(filter(None, sub.values.flatten()))
            blocks.append(sub)
            col+=3
            i+=1
    return blocks
content_blocks(grid)[6]

{2.0, 6.0, 8.0}

### Define a class Cell with a value and block number

In [31]:
class Cell():
    def __init__(self, grid, x, y):
        '''grid is a pd.dataFrame
        x and y are integers within 0 and 8'''
        self.x=x
        self.y=y
        self.value = grid.iloc[x][y]
        self.id=str(x)+str(y)
    def find_block(self):
        if self.x<3:
            self.list_grids=[0,1,2]
        elif self.x<6:
            self.list_grids=[3,4,5]
        else:
            self.list_grids=[6,7,8]
        if self.y<3:
            return self.list_grids[0]
        elif self.y<6:
            return self.list_grids[1]
        else:
            return self.list_grids[2]
ex_cell=Cell(grid,5,8)
print(ex_cell.value)
print(ex_cell.find_block())

2.0
5


### Algorithms
[Reference](https://www.kristanix.com/sudokuepic/sudoku-solving-techniques.php)
- 1 <strong>Sole candidate</strong>: cells that can only be filled by one digit (in a row + column + block)
- 2 <strong>Unique candidate</strong>: the cell has several options but is the only one in the row/column/block to have one digit as an option
- 3 <strong>Block and column/row interaction</strong>: if a digit can only be on 1 row or column on a block then we can eliminate it from the possibilities of the other blocks for that row or column.
- 4 <strong>Block/block interaction</strong>: if on 2 aligned blocks a digit can be in only 2 rows then we can eliminate it from these two rows on the third block
- 5 <strong>Naked subset</strong>: when 2 cells in a row/column/block are the only ones to admit the same only two digits, even if these digits are a possibility on other cells (so they can be removed from the options for these other cells, and the list of options for the 2 cells are reduced to these digits only)
- 6 <strong>X-Wing</strong>: look at cells disposed at corners of a rectangle of any size (each corner in a different block): if one digit is an option in each corner cell (and also on their respective rows (or columns)), and if in addition the digit can only be in one of the corners on each side of the rectangle: then the digit can only be in the corner and can be removed from the options for other cells in the rectangle. More generally, <strong>swordfish</strong>: when candidate cells for a specific digit happen to allow for this option on separate rows or columns, but specifically 2 times on each row & 2 times on each column; and at the same time the digit is an option only in all of the rows (or all of the columns). Then it can be removed from all the columns (or rows). To recognize this pattern, see if it is possible to loop (using only vertical or horizontal lines) between all the cells containing a candidate number.
- 7 <strong>Forcing chain</strong>: guess an option for a cell and write down how it would affect other cells. Then guess another option and see if other cells would resolve in the same digit anyway.

In [32]:
def unique_candidate(grid,cell,dict_possibles,filled_with_possibles):
    forbidden=[]
    if cell.id in dict_possibles:
        possible=dict_possibles[cell.id]
    else:
        possible=list(range(1,10))
    #list digits in the row
    forbidden=set(filter(None,grid.loc[cell.x]))
    #list digits in the column
    forbidden.update(filter(None,set(list(grid[grid.columns[cell.y]]))))
    #list digits in the subgrid
    forbidden.update(content_blocks(grid)[cell.find_block()])
    #remove forbidden digits from a set of all possible digits (from 1 to 9)
    possible=set(possible).difference(forbidden)
    if len(possible)==1:
        grid.iloc[cell.x][cell.y]=list(possible)[0]
    else:
        dict_possibles[cell.id]=possible
        filled_with_possibles.iloc[cell.x][cell.y]=list(possible)
        if possible==set():
            print('Error, this grid cannot be solved: check the grid pre-filled with options, one cell has no options left')
            return 1   
#unique_candidate(grid,Cell(grid,2,5))

In [33]:
def solve(grid):
    dict_possibles={}
    filled_ori=len(list(filter(None,grid.values.flatten())))
    print(filled_ori)
    filled=0
    while filled<81:
        filled_with_possibles=grid.copy()
        for row in range(0,9):
            for col in range(0,9):
                alert=False
                cell=Cell(grid,row,col)
                if cell.value=='':
                    result=unique_candidate(grid,cell,dict_possibles,filled_with_possibles)
                    if result==1:
                        return dict_possibles, filled_with_possibles
        if filled==len(list(filter(None,grid.values.flatten()))):
            print("Sorry, cannot solve this puzzle, go to the hardcore solution")
            return dict_possibles, filled_with_possibles
        filled=len(list(filter(None,grid.values.flatten())))
        print(filled)
    return grid,0

In [34]:
test=grid.copy()
dict_possibles, filled_with_possibles=solve(test)

28
34
41
47
51
Error, this grid cannot be solved: check the grid pre-filled with options, one cell has no options left


In [35]:
filled_with_possibles

,0,1,2,3,4,5,6,7,8
0,3,1,"[8, 6]",2,[],,,,4
1,7,4,,3,6,,,,5
2,9,5,,,,,,2,
3,2,3,5,1,8,6,4,7,9
4,1,6,9,,,,5,3,8
5,4,8,7,9,5,3,6,1,2
6,5,2,1,,7,8,9,4,3
7,6,9,,,1,,,,7
8,8,7,,,9,,,,


In [36]:
test

,0,1,2,3,4,5,6,7,8
0,3,1,,2,,,,,4
1,7,4,,3,6,,,,5
2,9,5,,,,,,2,
3,2,3,5,1,8,6,4,7,9
4,1,6,9,,,,5,3,8
5,4,8,7,9,5,3,6,1,2
6,5,2,1,,7,8,9,4,3
7,6,9,,,1,,,,7
8,8,7,,,9,,,,


In [37]:
# def solve_hardcore(grid):
#     '''links the possibilities in one cell to all the possibilities in the row, column and block'''
#     dict_possibles={}
#     filled_ori=len(list(filter(None,grid.values.flatten())))
#     print(filled_ori)
#     filled=0
#     while filled<81:
#         for row in range(0,9):
#             for col in range(0,9):
#                 cell=Cell(grid,row,col)
#                 if cell.value=='':    
#                     solve_for_cell(grid,cell,dict_possibles)
#         if filled!=len(list(filter(None,grid.values.flatten()))):
#             filled=len(list(filter(None,grid.values.flatten())))
#             print(filled)
#             continue;
#         else:
#             return print("Sorry, tried my best, this puzzle is pure evil. Good luck!")   
#             break;
#     return grid